In [1]:
import sys
sys.path.append("/Users/rachelcohen/hogtie/hogtie")

In [2]:
import hogtie
hogtie.set_loglevel("INFO")

In [3]:
import subprocess
import toytree
import numpy as np

In [6]:
TREE = toytree.rtree.baltree(ntips=10, treeheight=10)
DATA = np.array([0, 0, 0, 1, 1, 1, 1, 1, 0, 1]).astype(float)

TREE.draw()

(<toyplot.canvas.Canvas at 0x7f8078833880>,
 <toytree.Render.ToytreeMark at 0x7f8078828ca0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t473da617d60146e7b591d59a629fcfdc"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

In [20]:
tree = toytree.rtree.unittree(ntips = 10)
tree.draw()

(<toyplot.canvas.Canvas at 0x7f807aa19df0>,
 <toytree.Render.ToytreeMark at 0x7f807a4baa90>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="260.0px" height="275.0px" viewBox="0 0 260.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7f4946e049a74a41b47f684a0098c08e"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

In [42]:
model = hogtie.BinaryStateModel(TREE, DATA)
model.optimize()

10:00 | INFO    | optimize        | {'alpha': 1.025604, 'beta': 0.584925, 'Lik': -0.002575, 'negLogLik': 5.96, 'convergence': True}


In [43]:
model.log_lik

5.96

In [44]:
model.tree.get_node_values('likelihood',True,True)

array([[0.48181443, 0.51818557],
       [0.30632235, 0.69367765],
       [0.65678295, 0.34321705],
       [0.58701247, 0.41298753],
       [0.11307941, 0.88692059],
       [0.16262517, 0.83737483],
       [0.94785669, 0.05214331],
       [0.05444168, 0.94555832],
       [0.9772489 , 0.0227511 ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

In [46]:
out = subprocess.run([
    "Rscript", 
    "-e", "library(ape)",
    "-e", f"tree <- read.tree(text='{TREE.newick}')",
    "-e", f"data <- c{tuple(DATA[::-1])}",
    "-e", "object <- ace(data, tree, marginal=TRUE, type='discrete', model='ARD', use.expm=FALSE)",
    "-e", "print(object)",
    "-e", "print(object$lik.anc)",
    ], capture_output=True,
)

print(out.stdout.decode())


    Ancestral Character Estimation

Call: ace(x = data, phy = tree, type = "discrete", model = "ARD", use.expm = FALSE, 
    marginal = TRUE)

    Log-likelihood: -5.268686 

Rate index matrix:
  0 1
0 . 2
1 1 .

Parameter estimates:
 rate index estimate std-err
          1   0.5849  0.4291
          2   1.0256  0.6947

Scaled likelihoods at the root (type '...$lik.anc' to get them for all nodes):
        0         1 
0.4818151 0.5181849 
               0          1
 [1,] 0.48181510 0.51818490
 [2,] 0.30632691 0.69367309
 [3,] 0.58700949 0.41299051
 [4,] 0.11308474 0.88691526
 [5,] 0.05444436 0.94555564
 [6,] 0.65677934 0.34322066
 [7,] 0.16263154 0.83736846
 [8,] 0.94785282 0.05214718
 [9,] 0.97724724 0.02275276

